## Find then Tile for the GEMS image

- author : Sylvie Dagoret-Campagne
- creation date : 2025-05-24

In [ ]:
import astroquery
print(astroquery.__version__)

https://stackoverflow.com/questions/59699193/how-to-obtain-2d-cutout-of-an-image-from-a-skycoord-position

In [ ]:
# For Angle conversion
from astropy.coordinates import Angle
import astropy.units as u
from astropy.coordinates import SkyCoord
import pandas as pd
import numpy as np
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D

In [ ]:
from IPython.display import Image, display

## Configuration

### Target in ECDFS

- path to article : https://arxiv.org/pdf/1104.0931
- visual selection of the tiles : https://archive.stsci.edu/prepds/gems/browser.html
- path file download : https://archive.stsci.edu/pub/hlsp/gems/v_mk1/

In [ ]:
fig_table1 = "input_figs/table1_gemcandidates.png"
fig_table2 = "input_figs/table2_gemcandidates.png"

In [ ]:
Image(url= fig_table1,width=800)

In [ ]:
Image(url= fig_table2,width=500)

In [ ]:
#15422 44 03:32:38.21 –27:56:53.2 
ra1 = "03:32:38.21 hours"
dec1 = "-27:56:53.2 degrees"
tile1 = 44

#34244 94 03:32:06.45 –27:47:28.6 
ra2 = "03:32:06.45 hours"
dec2 = "-27:47:28.6 degrees"
tile2 = 94
# Je ne trouve pas cette tile
tile2 = 32

#40173 35 03:33:19.45 –27:44:50.0 
ra3 = "03:33:19.45 hours"
dec3 = "-27:44:50.0 degrees"
tile3 = 35

#43242 45 03:31:55.35 –27:43:23.5 
ra4 = "03:31:55.35 hours"
dec4 = "-27:43:23.5 degrees"
tile4 = 45

#46446 47 03:31:35.94 –27:41:48.2 
ra5 = "03:31:35.94 hours"
dec5 = "-27:41:48.2 degrees"
tile5 = 47

#12589 03:31:24.89 −27:58:07.0
ra6 = "03:31:24.89 hours"
dec6 = "-27:58:07.0 degrees"
tile6 = 17

#43797 03:31:31.74 −27:43:00.8 
ra7 = "03:31:31.74 hours"
dec7 = "-27:43:00.8 degrees"
tile7 = 47

#28294 03:31:50.54 −27:50:28.4 
ra8 = "03:31:50.54 hours"
dec8 = "-27:50:28.4 degrees"
tile8 = 33

#36857 03:31:53.24 −27:46:18.9
ra9 = "03:31:53.24 hours"
dec9 = "-27:46:18.9 degrees"
tile9 = 38

#36714 03:32:59.78 −27:46:26.4 
ra10 = "03:32:59.78 hours"
dec10 = "-27:46:26.4 degrees"
tile10 = 37


In [ ]:
lsstcomcam_targets = {}
# high rank
lsstcomcam_targets["ECDFS_G15422"] = {"field_name": "GEMS-15422", "ra": 53.159208333333325, "dec": -27.94811111111111,"tile":tile1}
lsstcomcam_targets["ECDFS_G34244"] = {"field_name": "GEMS-34244", "ra": 53.02687499999999 , "dec": -27.79127777777778,"tile":tile2}
lsstcomcam_targets["ECDFS_G40173"] = {"field_name": "GEMS-40173", "ra": 53.33104166666666 , "dec": -27.747222222222224,"tile":tile3}
lsstcomcam_targets["ECDFS_G43242"] = {"field_name": "GEMS-43242", "ra": 52.980624999999996 , "dec": -27.72319444444444,"tile":tile4}
lsstcomcam_targets["ECDFS_G46446"] = {"field_name": "GEMS-46446", "ra": 52.89975 , "dec": -27.696722222222224,"tile":tile5}

# low rank
lsstcomcam_targets["ECDFS_G12589"] = {"field_name": "GEMS-12589", "ra": 52.85370833333333, "dec": -27.96861111111111,"tile":tile6}
lsstcomcam_targets["ECDFS_G43797"] = {"field_name": "GEMS-43797", "ra": 52.88224999999999, "dec": -27.71688888888889,"tile":tile7}

lsstcomcam_targets["ECDFS_G28294"] = {"field_name": "GEMS-28294", "ra": 52.960583333333325 , "dec": -27.84122222222222,"tile":tile8}
lsstcomcam_targets["ECDFS_G6857"] = {"field_name": "GEMS-6857", "ra": 52.97183333333333 , "dec": -27.771916666666666,"tile":tile9}
lsstcomcam_targets["ECDFS_G36714"] = {"field_name": "GEMS-36714", "ra": 53.249083333333324, "dec": -27.773999999999997,"tile":tile10}


In [ ]:
df = pd.DataFrame(lsstcomcam_targets).T

In [ ]:
# candidates
#key = "ECDFS_G15422"
key = "ECDFS_G34244"
#key = "ECDFS_G40173"
#key= "ECDFS_G43242"
#key= "ECDFS_G46446"

# unknown
#key = "ECDFS_G12589"
#key = "ECDFS_G43797"
#key = "ECDFS_G28294"
#key = "ECDFS_G6857"
#key = "ECDFS_G36714"

the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_name = the_target["field_name"]
tile_num = the_target["tile"]

target_title = (
    the_target["field_name"] + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)

# Exemple : coordonnées au centre du champ GEMS
coord = SkyCoord(ra=target_ra, dec=target_dec, unit=(u.deg, u.deg))

In [ ]:
tile_num

### Access to remote files

- visual selection of the tiles : https://archive.stsci.edu/prepds/gems/browser.html
- path file download : https://archive.stsci.edu/pub/hlsp/gems/v_mk1/

### Config

- http://archive.stsci.edu/pub/hlsp/gems/v_mk1/h_gems_v35_mk1.fits
- url = f"http://archive.stsci.edu/pub/hlsp/gems/z_mk1/h_gems_z{tile_num}_mk1.fits"

In [ ]:
# select your bands
SELECT_VBAND = "v"
SELECT_ZBAND = "z"
SELECT_BAND = SELECT_ZBAND

## Try to find tile number

In [ ]:
pixel_width = 120

In [ ]:
for tile_num in range(1,64):

    stile = str(tile_num).zfill(2)
    #define the input url¶
    url = f"http://archive.stsci.edu/pub/hlsp/gems/{SELECT_BAND}_mk1/h_gems_{SELECT_BAND}{stile}_mk1.fits"
    # load image

    try:
        hdul = fits.open(url)
        print(f"{stile} ==> keep {url}")
    except:
        print(f"{stile} ==> skip {url}")
        continue
        
    hdu = hdul[0]
    data = hdu.data
    header = hdu.header
    wcs = WCS(header)

    #
    cutout_header = hdu.header.copy()

    # Create a 2D WCS specifically for the spatial cutout
    wcs_full = WCS(cutout_header)
    wcs_2d = wcs_full.celestial

    # Extract only the 2D spatial slice of the data
    data_2d = data
    size = (pixel_width, pixel_width)

    # Create the cutout with the 2D WCS
    try:
        cutout = Cutout2D(data_2d, position=coord, size=size, wcs=wcs_2d)
        print(f"tile OK :  {stile}")
        break
    except:
        print(f"tile skipped :  {stile}")
        continue
        
        
